In [8]:
import os, sys, re
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# установка стиля matplotlib
%matplotlib inline
plt.style.use('ggplot')

# доступные стили отображения графика в matplotlib
# plt.style.available

# библиотека иерархической кластеризации (для построения дендрограммы) 
from scipy.cluster import hierarchy

# библиотека для построения выпуклой оболочки множества точек 
from scipy import spatial

# from ydata_profiling import ProfileReport         # отключено пока не "починят"
import sweetviz as sv
import dtale
import statistics as stcs
import statsmodels.stats.proportion as stsmdls

# библиотека для расчетов корреляций
from scipy import stats
from sklearn import metrics

# библиотека для нормализации, стандартизации
from sklearn import preprocessing

# библиотека модели линейной регрессии
from sklearn import linear_model

# библиотека модели деревьев решения
from sklearn import tree

# библиотека модели ансамблей
from sklearn import ensemble

# библиотека модели кластеризации
from sklearn import cluster

# библиотека модели эллиптической кластеризации
from sklearn import mixture

# библиотека модели метода главных компонент PCA
from sklearn import decomposition

# библиотека модели TSNE
from sklearn import manifold

# библиотека clustergram-кластеризации
from clustergram import Clustergram

# библиотека для разделения датасета
from sklearn import model_selection

# библиотека для исключения признаков RFE
from sklearn import feature_selection

# библиотека для кодирования признаков
import category_encoders as ce

# для обработки timestamp в формате utc
import datetime, time

# работа с json
import json
from pprint import pprint

# веб-скрэпинг сайтов
import requests
from bs4 import BeautifulSoup

# выводить все результаты вычислений в ячейках Code
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# подключить Latex
from IPython.display import Latex

# загрузка файлов
import wget

# логгирование / логирование
import logging

# скрытие сообщений об ошибках при обработке ответов
import warnings
# игнорируем предупреждения
warnings.filterwarnings('ignore')

# изменить формат вывода pandas
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
# pd.set_option('display.max_info_rows', 500)
# pd.set_option('display.max_info_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.large_repr', 'truncate')

In [2]:
def get_metrics(X_train:pd.DataFrame, y_train:pd.Series, X_test:pd.DataFrame, y_test:pd.Series) -> float:
    """ Вычисление метрик MAE, MAPE, MSE, RMSE, R2-score

    Args:
        X_train (pd.DataFrame): тренировочный датасет
        y_train (pd.Series): тренировочный целевой вектор
        X_test (pd.DataFrame): тестовый датасет
        y_test (pd.Series): тестовый целевой вектор

    Returns:
        float: метрики MAE, MAPE, MSE, RMSE, R2-score
    """
    # присваиваем модели тип - линейную регрессию, обучаем её, предсказываем значения на тестовой выборке
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)

    # вычисление метрик, округление до 3 знака после запятой и перевод к типу float
    mae = float(np.round(metrics.mean_absolute_error(y_test, y_test_pred), 3))
    mape = float(np.round(metrics.mean_absolute_percentage_error(y_test, y_test_pred), 3))
    mse = float(np.round(metrics.mean_squared_error(y_test, y_test_pred), 3))
    rmse = float(np.round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), 3))
    r2 = float(np.round(metrics.r2_score(y_test, y_test_pred), 3))

    # возвращаем кортеж с метриками
    return mae, mape, mse, rmse, r2

# <center> Практика 9.5

In [3]:
# выгружаем файл, перезаписываем при наличии, выводим сообщения только при ошибке
!wget -q -O ./data_ford_price.xlsx https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [4]:
# читаем исходный датасет
data = pd.read_excel('data_ford_price.xlsx')
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [6]:
## предобработка данных
# обработаем пропуски - просто удалим их
data.dropna(inplace = True)

#округлим признак lat и long до целочисленного
data['lat']=round(data['lat'], 0)
data['long']=round(data['long'], 0)

# выделим признак 'condition' как 'object', т.к. он НЕ номинальный, а порядковый числовой признак
data['condition'] = data['condition'].astype('object')

# оставляем только числовые признаки
num_cols = [col for col in data.columns if data[col].dtypes != 'object']
data_num = data[num_cols].copy()
data_num.head()

,price,year,cylinders,odometer,lat,long,weather
0,43900,2016,6,43500,36.0,-82.0,59.0
1,15490,2009,8,98131,40.0,-74.0,52.0
2,2495,2002,8,201803,42.0,-83.0,45.0
3,1300,2000,8,170305,41.0,-82.0,49.0
5,6995,2003,8,167662,46.0,-123.0,50.0


In [9]:
# выделяем целевой признак
y_num = data_num['price']
X_num = data_num.drop(columns='price')

# разделяем данные на тренировочную и тестовую выборки
X_train_num, X_test_num, y_train_num, y_test_num = model_selection.train_test_split(
    X_num, y_num, 
    test_size=0.2, 
    random_state=42
)

In [10]:
## обучение модели линейной регрессии на числовых признаках
## выводим метрики
num_metrics = get_metrics(X_train_num, y_train_num, X_test_num, y_test_num)
print(f'MAE score: {num_metrics[0]}\nR2 score: {num_metrics[4]}')

MAE score: 4621.384
R2 score: 0.634


In [11]:
## выбираем лучшие признаки по методу рекурсивного исключения признаков (RFE)
estimator = linear_model.LinearRegression();
selector_rfe = feature_selection.RFE(estimator, n_features_to_select=3, step=1);
selector_rfe.fit(X_train_num, y_train_num);

# выводим отобранные признаки
rfe_cols = list(selector_rfe.get_feature_names_out())
rfe_cols

RFE(estimator=LinearRegression(), n_features_to_select=3)

['year', 'cylinders', 'lat']

In [12]:
## выбираем лучшие признаки по методу "выбор k лучших переменных" (SelectKBest)
selector_skb = feature_selection.SelectKBest(feature_selection.f_regression, k=3);
selector_skb.fit(X_train_num, y_train_num);

# выводим отобранные признаки
skb_cols = list(selector_skb.get_feature_names_out())
skb_cols

SelectKBest(k=3, score_func=<function f_regression at 0x127afd2d0>)

['year', 'cylinders', 'odometer']

In [13]:
## обученение регрессии на RFE-признаках
# выбираем RFE-признаки из данных
X_train_rfe = X_train_num[rfe_cols]
X_test_rfe = X_test_num[rfe_cols]

## выводим метрики
fre_metrics = get_metrics(X_train_rfe, y_train_num, X_test_rfe, y_test_num)
print(f'MAE score: {fre_metrics[0]}\nR2 score: {fre_metrics[4]}')

MAE score: 5107.732
R2 score: 0.573


In [14]:
## обученение регрессии на выбранных k лучших столбцах
# выбираем k-признаки из данных
X_train_skb = X_train_num[skb_cols]
X_test_skb = X_test_num[skb_cols]

## выводим метрики
skb_metrics = get_metrics(X_train_skb, y_train_num, X_test_skb, y_test_num)
print(f'MAE score: {skb_metrics[0]}\nR2 score: {skb_metrics[4]}')

MAE score: 4627.369
R2 score: 0.631


#### **Вывод:**

Средняя абсолютная ошибка (Mean Absolute Error) - это число показывает, насколько в среднем обученная модель ошибается, чем меньше значение метрики, тем лучше качество модели.

Таким образом, у нас есть метрика MAE, рассчитанная для 3 моделей линейной регрессии:
* модель линейной регрессии для всех числовых признаков;

* модель линейной регрессии для признаков `['year', 'cylinders', 'lat']`, полученных методом рекурсивного исключения признаков (RFE);

* модель линейной регрессии для признаков `['year', 'cylinders', 'odometer']`, полученных методом выбор k лучших признаков (SelectKBest).

Интересуют модели *вторая* и *третья*:

* RFE MAE составляет - 5107.732

* SelectKBest MAE составляет - 4627.369

Полученные показатели позволяют сделать вывод о том, что качество третьей модели лучше второй, поскольку MAE, полученная на k-признаках, меньше MAE, рассчитанной для модели, обученной на RFE-признаках. Кроме того, метрика $R^2$, которая показывает, какую долю разнообразия (дисперсии) смогла уловить модель в данных, выше на третьей модели (0.631 против 0.573).

RFE-модель проигрывает SelectKBest-модели. SKB считывает результаты по всем корреляциям и разом убирает все, кроме трёх сильнейших. RFE итеративно исключает по одному слабейшему признаку за каждый проход, пока не останется три самых сильных.

Если сравнивать результаты *третьей* и *первой* моделей, то, безусловно, в первой модели количество числовых признаков превалирует и вносит дополнительную, хоть и незначительную, детализацию модели, построенную на всех числовых признаках, и, как следствие, выигрыш модели на всех числовых признаках несёт положительный отрыв, пусть и скромный.